In [ ]:
!nvidia-smi

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Colab/kaggle/kaggle.json ~/.kaggle

!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c chaii-hindi-and-tamil-question-answering

!unzip chaii-hindi-and-tamil-question-answering.zip
!rm chaii-hindi-and-tamil-question-answering.zip

In [ ]:
!pip install -U pytorch-lightning transformers

In [12]:
import os
import random

import numpy as np
import pandas as pd

# import torch
# import torch.nn as nn
# from torch.nn import functional as F
# from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim

# import pytorch_lightning as pl
# from pytorch_lightning import Trainer, seed_everything
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
# from pytorch_lightning.loggers import WandbLogger

import sys
import argparse
import json
import nltk
from tqdm import tqdm
import zipfile

## MLQA

In [11]:
!wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip

--2021-09-05 07:17:00--  https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75719050 (72M) [application/zip]
Saving to: ‘MLQA_V1.zip’

MLQA_V1.zip         100%[===================>]  72.21M  19.9MB/s    in 4.0s    

2021-09-05 07:17:04 (18.1 MB/s) - ‘MLQA_V1.zip’ saved [75719050/75719050]



In [13]:
with zipfile.ZipFile('/content/MLQA_V1.zip') as zip_ref:
    zip_ref.extractall('/content/')

In [14]:
mlqa_train_data = "/content/MLQA_V1/dev/dev-context-hi-question-hi.json"
mlqa_test_data = "/content/MLQA_V1/test/test-context-hi-question-hi.json"

with open(mlqa_train_data, 'r') as file_input:
    train_file = json.load(file_input)
    
with open(mlqa_test_data, 'r') as file_input:
    test_file = json.load(file_input)

In [16]:
def preprocess(dataset, tier):
    num_exs = 0 
    examples = []

    for articles_id in tqdm(range(len(dataset['data'])), desc="Preprocessing {}".format(tier)):
        article_paragraphs = dataset['data'][articles_id]['paragraphs']
        for pid in range(len(article_paragraphs)):
            context = article_paragraphs[pid]['context']
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')
            qas = article_paragraphs[pid]['qas'] 
            for qn in qas:
                question = qn['question'] 
                ans_text = qn['answers'][0]['text']
                ans_start_charloc = qn['answers'][0]['answer_start']
                ans_end_charloc = ans_start_charloc + len(ans_text)
                examples.append(
                    {
                        # 'id':articles_id,
                        'context':context, 
                        'question':question, 
                        'answer_text':ans_text, 
                        'answer_start':ans_start_charloc, 
                        # 'answer_end':ans_end_charloc
                    }
                )

                num_exs += 1
    print(num_exs)    
    return examples

In [17]:
examples_train = preprocess(train_file, 'dev')
examples_test = preprocess(test_file, 'test')

Preprocessing dev: 100%|██████████| 217/217 [00:00<00:00, 50360.43it/s]


507


Preprocessing test: 100%|██████████| 2038/2038 [00:00<00:00, 92466.70it/s]

4918


In [20]:
examples = examples_train + examples_test
mlqa = pd.DataFrame(examples)
mlqa['language'] = 'hindi'
mlqa.head()

,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi


## XQUAD

In [21]:
!git clone https://github.com/deepmind/xquad.git

Cloning into 'xquad'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 12), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [22]:
xquad_train_file = '/content/xquad/xquad.hi.json'

with open(xquad_train_file, 'r') as file_input:
    train_file = json.load(file_input)
    
examples_train = preprocess(train_file, 'dev')
xquad = pd.DataFrame(examples_train)
xquad['language'] = 'hindi'
xquad.head()

Preprocessing dev: 100%|██████████| 48/48 [00:00<00:00, 16791.21it/s]

1190


,context,question,answer_text,answer_start,language
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35,hindi
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380,hindi
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743,hindi
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90,hindi
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169,hindi


## save external data

In [23]:
# mlqa.to_csv("/content/drive/MyDrive/Colab/kaggle/chaii-hindi-and-tamil-question-answering/input/external_data/mlqa_hindi.csv", index=False)
# xquad.to_csv("/content/drive/MyDrive/Colab/kaggle/chaii-hindi-and-tamil-question-answering/input/external_data/xquad_hindi.csv", index=False)